In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/MyDrive/Remi/

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1WJLnwO7mUn1QFbhFIJ6fMZjZWuCKf1ub/Remi


In [ ]:
#!pip install --upgrade tensorflow-gpu
#Install TF-Hub.
!pip install tensorflow-hub
!pip install seaborn

!pip3 install tensorflow-text --force-reinstall
import tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 4.6 MB/s 
     |████████████████████████████████| 108 kB 49.6 MB/s 
     |████████████████████████████████| 578.0 MB 16 kB/s 
     |████████████████████████████████| 4.7 MB 37.1 MB/s 
     |████████████████████████████████| 15.7 MB 33.9 MB/s 
     |████████████████████████████████| 123 kB 53.1 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 1.2 MB 47.6 MB/s 
     |████████████████████████████████| 14.1 MB 40.5 MB/s 
     |████████████████████████████████| 57 kB 6.0 MB/s 
     |████████████████████████████████| 40 kB 6.4 MB/s 
     |████████████████████████████████| 4.1 MB 39.8 MB/s 
     |██████

In [ ]:
# !pip install dask
# !pip install dask[dataframe] 
!pip install scispacy

import numpy as np
import torch
import tensorflow
import pandas as pd
import os
import json
import time
import glob
import re
import sys
import collections
from nltk import flatten
# import dask
# from dask import delayed,compute
# import dask.dataframe as dd
# from dask.multiprocessing import get
from tqdm._tqdm_notebook import tqdm_notebook
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm_notebook.pandas()
from nltk.tokenize import word_tokenize
from collections import defaultdict

import nltk
nltk.download('punkt')
import scispacy
import spacy
import scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.3 MB/s 
     |████████████████████████████████| 71 kB 5.3 MB/s 
     |████████████████████████████████| 13.5 MB 24.2 MB/s 
     |████████████████████████████████| 188 kB 52.0 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.3.0
    Uninstalling typing-extensions-4.3.0:
      Successfully uninstalled typing-extensions-4.3.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


ImportError: ignored

## Data loading and pre-processing

In [ ]:
import re
import math

i = 0
dfs = []
dir = 'wiki_annotation_pages/nl/'
for filename in os.listdir(dir):
    data = pd.read_json(dir + filename)  
    data = data["query"]["pages"][0]
    filename = filename.replace('.json', '')
    data.update({'filename': filename})
    data = pd.DataFrame([data])
    dfs.append(data)

temp = pd.concat(dfs, ignore_index=True)
temp.rename(columns={'title':'Subject'}, inplace=True)
temp.rename(columns={'extract':'content'}, inplace=True)

df_wikipedia =temp[['Subject','content', 'filename']]

#df_wikipedia.content =df_wikipedia.content.replace(to_replace='<[^<]+?>',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='<p(.*?)>',value='<p>',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='<link(.*?)>',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='-',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='  ',value='',regex=True)
df_wikipedia.content =df_wikipedia.content.apply(lambda x:x.strip())

# df_wikipedia.to_csv('df_wiki.csv')
#print(df_wikipedia.content)

## data cleaning for subject data 
df_wikipedia.Subject =df_wikipedia.Subject.replace(to_replace='[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
df_wikipedia.Subject =df_wikipedia.Subject.replace(to_replace='\s+',value=' ',regex=True)
df_wikipedia.Subject =df_wikipedia.Subject.replace(to_replace='  ',value='',regex=True)
df_wikipedia.Subject =df_wikipedia.Subject.apply(lambda x:x.strip())
df_wikipedia.content =df_wikipedia.content.replace(to_replace='<p> </p>',value='',regex=True)

df_wikipedia["content"]=df_wikipedia["content"].str.split("\n<p>")
df_wikipedia = df_wikipedia.explode("content").reset_index(drop=True)

#print(df_wikipedia["content"].str.split("</p>\s+<h")[5])
df_wikipedia["content"]=df_wikipedia["content"].str.split("</p>\s+<h")
df_wikipedia = df_wikipedia.explode("content").reset_index(drop=True)

df_wikipedia["content"]=df_wikipedia["content"].str.split("\n</p>\n")
df_wikipedia = df_wikipedia.explode("content").reset_index(drop=True)

df_wikipedia["content"]=df_wikipedia["content"].str.split("</li></ul><h3(.*?)>")
df_wikipedia = df_wikipedia.explode("content").reset_index(drop=True)

#print(df_wikipedia["content"][1014])

df_wikipedia["content"]=df_wikipedia["content"].str.split("</h3>\s+<ul><li>")
df_wikipedia = df_wikipedia.explode("content").reset_index(drop=True)

df_wikipedia['content']=[entry.lower() for entry in df_wikipedia['content']]
SW=['<span>', '<li>', '</li>', '<ul>', '</ul>', '<b>', '</b>']
for sw in SW:
    df_wikipedia.content=df_wikipedia.content.replace(to_replace=sw,value='',regex=True)

df_wikipedia.content =df_wikipedia.content.replace(to_replace='\n',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='<\/p>\s+<h(.*?)</h(.*?)>',value='',regex=True)  

df_wikipedia = df_wikipedia[df_wikipedia.content != ' ']
df_wikipedia.content =df_wikipedia.content.replace(to_replace='[!"#$%&\'()*+,:;=?@[\\]^_`{|}~]',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='<[^<]+?>',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='\s+',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='  ',value=' ',regex=True)

df_wikipedia = df_wikipedia[~df_wikipedia['content'].str.startswith('2>')]
df_wikipedia = df_wikipedia[~df_wikipedia['content'].str.startswith('3>')]
df_wikipedia = df_wikipedia[~df_wikipedia['content'].str.startswith('4>')]
df_wikipedia = df_wikipedia[~df_wikipedia['content'].str.startswith('5>')]

df_wikipedia = df_wikipedia.reset_index(drop=True)
df_wikipedia = df_wikipedia.loc[df_wikipedia['content'].str.len() > 1]
df_wikipedia = df_wikipedia.reset_index(drop=True)

df_wikipedia["tokenized"] = df_wikipedia["content"]

# df_wikipedia = df_wikipedia[df_wikipedia['content'].notna()]
print('----------')
#print(len(df_wikipedia['content'][28]))
df_wikipedia.to_csv('df_wiki.csv')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


----------


### Word Tokenization

In [ ]:
df_wikipedia['Word tokenize']= [word_tokenize(entry) for entry in df_wikipedia.tokenized]

### Word Lemmatization

In [ ]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
def wordLemmatizer(data):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    file_clean_k =pd.DataFrame()
    for index,entry in enumerate(data):
        
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if len(word)>1 and word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
            # The final processed set of words for each iteration will be stored in 'text_final'
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                #file_clean_k=file_clean_k.replace(to_replace ="\[.", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ="'", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace =" ", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ='\]', value = '', regex = True)
    return file_clean_k

In [ ]:
df_wikipedia.shape 

(829, 5)

In [ ]:
df_wikipedia

,Subject,content,filename,tokenized,Word tokenize
0,Coronacrisis in Nederland,de coronacrisis in nederland is onderdeel van...,Coronacrisis_in_Nederland,de coronacrisis in nederland is onderdeel van...,"[de, coronacrisis, in, nederland, is, onderdee..."
1,Coronacrisis in Nederland,begin 2020 werd sars cov 2 – het coronavirus d...,Coronacrisis_in_Nederland,begin 2020 werd sars cov 2 – het coronavirus d...,"[begin, 2020, werd, sars, cov, 2, –, het, coro..."
2,Coronacrisis in Nederland,het maatschappelijke verkeer werd ontwricht do...,Coronacrisis_in_Nederland,het maatschappelijke verkeer werd ontwricht do...,"[het, maatschappelijke, verkeer, werd, ontwric..."
3,Coronacrisis in Nederland,begin 2021 werd een grootschalig vaccinatiepro...,Coronacrisis_in_Nederland,begin 2021 werd een grootschalig vaccinatiepro...,"[begin, 2021, werd, een, grootschalig, vaccina..."
4,Coronacrisis in Nederland,het meest relevante cijfer om het verloop van ...,Coronacrisis_in_Nederland,het meest relevante cijfer om het verloop van ...,"[het, meest, relevante, cijfer, om, het, verlo..."
...,...,...,...,...,...
824,CureVac,in 2007 curevac received the innovation prize ...,CureVac,in 2007 curevac received the innovation prize ...,"[in, 2007, curevac, received, the, innovation,..."
825,CureVac,by 2017 curevac had received approximately €30...,CureVac,by 2017 curevac had received approximately €30...,"[by, 2017, curevac, had, received, approximate..."
826,CureVac,on 11 march 2020 it was reported that curevac ...,CureVac,on 11 march 2020 it was reported that curevac ...,"[on, 11, march, 2020, it, was, reported, that,..."
827,CureVac,cvncov is an mrna vaccine that encodes a minim...,CureVac,cvncov is an mrna vaccine that encodes a minim...,"[cvncov, is, an, mrna, vaccine, that, encodes,..."


In [ ]:
import pandas as pd
import numpy as np
import os 
import re
import operator
import pickle
import nltk 
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
import os.path
import json
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')

df_clean = wordLemmatizer(df_wikipedia['Word tokenize'][0:989]) 
df_clean

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Keyword_final
0,"['de', 'coronacrisis', 'nederland', 'onderdeel..."
1,"['begin', 'werd', 'sars', 'cov', 'het', 'coron..."
2,"['het', 'maatschappelijke', 'verkeer', 'werd',..."
3,"['begin', 'werd', 'een', 'grootschalig', 'vacc..."
4,"['het', 'meest', 'relevante', 'cijfer', 'om', ..."
...,...
824,"['curevac', 'receive', 'innovation', 'prize', ..."
825,"['curevac', 'receive', 'approximately', 'milli..."
826,"['march', 'report', 'curevac', 'ag', 'ceo', 'd..."
827,"['cvncov', 'mrna', 'vaccine', 'encode', 'minim..."


In [ ]:
df_clean=df_clean.replace(to_replace ="\[.", value = '', regex = True)
df_clean=df_clean.replace(to_replace ="'", value = '', regex = True)
df_clean=df_clean.replace(to_replace =" ", value = '', regex = True)
df_clean=df_clean.replace(to_replace ='\]', value = '', regex = True)

### Added WordLemmatize words into given dataframe

In [ ]:
# df_wikipedia_save= df_clean
# df_wikipedia_save
df_wikipedia.insert(loc=3, column='Clean_Keyword', value=df_clean['Keyword_final'].tolist())
df_wikipedia_save= df_wikipedia
#df_wikipedia_save = df_wikipedia_save.drop(['Word tokenize,'Clean_Keyword'],axis=1)
#df_wikipedia = df_wikipedia.drop(['Word tokenize','Clean_Keyword'],axis=1)
df_wikipedia_save

,Subject,content,filename,Clean_Keyword,tokenized,Word tokenize
0,Coronacrisis in Nederland,de coronacrisis in nederland is onderdeel van...,Coronacrisis_in_Nederland,"de,coronacrisis,nederland,onderdeel,van,de,wer...",de coronacrisis in nederland is onderdeel van...,"[de, coronacrisis, in, nederland, is, onderdee..."
1,Coronacrisis in Nederland,begin 2020 werd sars cov 2 – het coronavirus d...,Coronacrisis_in_Nederland,"begin,werd,sars,cov,het,coronavirus,dat,covid,...",begin 2020 werd sars cov 2 – het coronavirus d...,"[begin, 2020, werd, sars, cov, 2, –, het, coro..."
2,Coronacrisis in Nederland,het maatschappelijke verkeer werd ontwricht do...,Coronacrisis_in_Nederland,"het,maatschappelijke,verkeer,werd,ontwricht,do...",het maatschappelijke verkeer werd ontwricht do...,"[het, maatschappelijke, verkeer, werd, ontwric..."
3,Coronacrisis in Nederland,begin 2021 werd een grootschalig vaccinatiepro...,Coronacrisis_in_Nederland,"begin,werd,een,grootschalig,vaccinatieprogramm...",begin 2021 werd een grootschalig vaccinatiepro...,"[begin, 2021, werd, een, grootschalig, vaccina..."
4,Coronacrisis in Nederland,het meest relevante cijfer om het verloop van ...,Coronacrisis_in_Nederland,"het,meest,relevante,cijfer,om,het,verloop,van,...",het meest relevante cijfer om het verloop van ...,"[het, meest, relevante, cijfer, om, het, verlo..."
...,...,...,...,...,...,...
824,CureVac,in 2007 curevac received the innovation prize ...,CureVac,"curevac,receive,innovation,prize,state,baden,w...",in 2007 curevac received the innovation prize ...,"[in, 2007, curevac, received, the, innovation,..."
825,CureVac,by 2017 curevac had received approximately €30...,CureVac,"curevac,receive,approximately,million,equity,i...",by 2017 curevac had received approximately €30...,"[by, 2017, curevac, had, received, approximate..."
826,CureVac,on 11 march 2020 it was reported that curevac ...,CureVac,"march,report,curevac,ag,ceo,daniel,menichella,...",on 11 march 2020 it was reported that curevac ...,"[on, 11, march, 2020, it, was, reported, that,..."
827,CureVac,cvncov is an mrna vaccine that encodes a minim...,CureVac,"cvncov,mrna,vaccine,encode,minimal,piece,coron...",cvncov is an mrna vaccine that encodes a minim...,"[cvncov, is, an, mrna, vaccine, that, encodes,..."


## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator

## Create Vocabulary
vocabulary = set()

for doc in df_wikipedia.Clean_Keyword:
    vocabulary.update(doc.split(','))

vocabulary = list(vocabulary)

# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=vocabulary,dtype=np.float32)

print(tfidf)

# Fit the TfIdf model
tfidf.fit(df_wikipedia.Clean_Keyword)

# Transform the TfIdf model
tfidf_tran=tfidf.transform(df_wikipedia.Clean_Keyword)

TfidfVectorizer(dtype=<class 'numpy.float32'>,
                vocabulary=['betrouwbare', 'contraception', 'dance',
                            'identiteitsbewijs', 'available', 'harmony',
                            'verrichten', 'msf', 'bliezen', 'tenacious',
                            'konden', 'eastern', 'jarenlange', 'hashtag',
                            'waldeyer', 'national', 'equality', 'spoke',
                            'similarity', 'veelvuldig', 'zuofeng', 'hardly',
                            'aidsstadium', 'minimaliseren', 'consortium',
                            'kritische', 'ziek', 'property', 'papr', 'huis', ...])


In [ ]:
### Save model
with open('tfid.pkl','wb') as handle:
    pickle.dump(tfidf_tran, handle)

### load model
t = pickle.load(open('tfid.pkl','rb'))

### Save Vacabulary
with open("vocab_wiki_files.txt", "w") as file:
    file.write(str(vocabulary))

### load Vacabulary
with open("vocab_wiki_files.txt", "r") as file:
    data2 = eval(file.readline())

In [ ]:
def gen_vector_T(tokens):
    Q = np.zeros((len(vocabulary)))
    x= tfidf.transform(tokens)
    for token in tokens[0].split(','):
        try:
            ind = vocabulary.index(token)
            Q[ind]  = x[0, tfidf.vocabulary_[token]]
        except:
            pass
    return Q

In [ ]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [ ]:
def cosine_similarity_T(k, query):
    #print("Cosine Similarity")
    preprocessed_query = re.sub("\W+", " ", query).strip()
    tokens = word_tokenize(str(preprocessed_query))
    q_df = pd.DataFrame(columns=['q_clean'])
    q_df.loc[0,'q_clean'] =tokens
    q_df['q_clean'] =wordLemmatizer(q_df.q_clean)
    q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
    q_df=q_df.replace(to_replace ="'", value = '', regex = True)
    q_df=q_df.replace(to_replace =" ", value = '', regex = True)
    q_df=q_df.replace(to_replace ='\]', value = '', regex = True)

    d_cosines = []

    query_vector = gen_vector_T(q_df['q_clean'])
    
    print(query_vector)
    for d in tfidf_tran.A:
        
        d_cosines.append(cosine_sim(query_vector, d))
                    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    #print("")
    d_cosines.sort()
    #print(out)
    a = pd.DataFrame()
    for i,index in enumerate(out):
        a.loc[i,'index'] = str(index)
        a.loc[i,'Subject'] = df_wikipedia['Subject'][index]
        a.loc[i,'content'] = df_wikipedia['content'][index]
    for j,simScore in enumerate(d_cosines[-k:][::-1]):
        a.loc[j,'Score'] = simScore
    return a

In [ ]:
%time cosine_similarity_T(10,'johnson and johnson vaccines derived from murdered babies. at least 2 other vaccines are not. if you take vaccines use those.')

[0. 0. 0. ... 0. 0. 0.]
CPU times: user 132 ms, sys: 87.2 ms, total: 219 ms
Wall time: 139 ms


,index,Subject,content,Score
0,750,COVID 19 vaccine,internationally the access to covid‑19 tools a...,0.340187
1,743,COVID 19 vaccine,additional types of vaccines that are in clini...,0.339199
2,734,COVID 19 vaccine,a covid‑19 vaccine is a vaccine intended to pr...,0.333803
3,742,COVID 19 vaccine,subunit vaccines present one or more antigens ...,0.328737
4,766,COVID 19 vaccine,protocol mrna 1273 p301 pdf . moderna. protoc...,0.319484
5,339,Chinese government response to COVID 19,in july 2020 the chinese government granted an...,0.317339
6,735,COVID 19 vaccine,prior to covid‑19 a vaccine for an infectious ...,0.293005
7,765,COVID 19 vaccine,vaccine protocols,0.286098
8,741,COVID 19 vaccine,inactivated vaccines consist of virus particle...,0.271578
9,749,COVID 19 vaccine,there have been several unique challenges with...,0.271332


## Google Sentence Encoder

In [ ]:
!pip install --upgrade tensorflow-gpu
#Install TF-Hub.
!pip install tensorflow-hub
!pip install seaborn

!pip3 install tensorflow-text --force-reinstall
import tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow_text-2.10.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)
  Using cached tensorflow-2.10.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (578.0 MB)
  Using cached h5py-3.7.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (4.1 MB)
  Using cached libclang-14.0.6-py2.py3-none-manylinux2010_x86_64.whl (14.1 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorflow_io_gcs_filesystem-0.27.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x

ImportError: ignored

In [ ]:
import pandas as pd
import numpy as np
import re, string
import os 
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import linear_kernel
import tensorflow_text


module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
model = hub.load(module_url)

# # #Create function for using modeltraining
def embed(input):
    return model(input)


Model_USE= embed(df_wikipedia['tokenized'])

exported = tf.train.Checkpoint(v=tf.Variable(Model_USE))
exported.f = tf.function(
    lambda  x: exported.v * x,
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])

tf.saved_model.save(exported,'TrainModel')

imported = tf.saved_model.load('TrainModel')
loadedmodel =imported.v.numpy()
loadedmodel.shape

def SearchDocument(query):
    q =[query]
    # embed the query for calcluating the similarity
    Q_Train =embed(q)

    # Calculate the Similarity
    linear_similarities = linear_kernel(Q_Train, loadedmodel).flatten() 
    #Sort top 10 index with similarity score
    Top_index_doc = linear_similarities.argsort()[:-11:-1]
    # sort by similarity score
    linear_similarities.sort()
    a = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        a.loc[i,'index'] = str(index)
        a.loc[i,'File_Name'] = df_wikipedia['content'][index] ## Read File name with index from File_data DF
    for j,simScore in enumerate(linear_similarities[:-11:-1]):
        a.loc[j,'Score'] = simScore
    return a

SearchDocument('Together with collaborators, we founded a spinout company to develop and commercialise a nasal spray vaccine against COVID. The vaccine is based on research by our and @UniEastFinland researchers. #HelsinkiHealth #corona #COVID19 #covidvaccine')

,index,File_Name,Score
0,741,inactivated vaccines consist of virus particle...,0.625157
1,746,a universal coronavirus vaccine is effective a...,0.589395
2,740,these vaccines are examples of non replicating...,0.584928
3,766,protocol mrna 1273 p301 pdf . moderna. protoc...,0.576884
4,745,intranasal vaccines target mucosal immunity in...,0.564301
5,739,several covid‑19 vaccines including the pfizer...,0.559850
6,742,subunit vaccines present one or more antigens ...,0.558546
7,734,a covid‑19 vaccine is a vaccine intended to pr...,0.557974
8,750,internationally the access to covid‑19 tools a...,0.557162
9,735,prior to covid‑19 a vaccine for an infectious ...,0.549555


## Sentence transformers

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import linear_kernel
from numpy import dot
from numpy.linalg import norm
import scipy

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

def embed(input):
    return model(input)

model.max_seq_length = 512
print("Max Sequence Length:", model.max_seq_length)
embeddings = model.encode(df_wikipedia['content'])

def SearchDocument(query):
    q =[query]
    # embed the query for calcluating the similarity
    Q_Train = model.encode(q)

    # Calculate the Similarity
    #linear_similarities = linear_kernel(Q_Train, embeddings).flatten()
    linear_similarities = scipy.spatial.distance.cdist(Q_Train, embeddings, "cosine")[0]
    linear_similarities = 1 - linear_similarities
    #Sort top 10 index with similarity score
    Top_index_doc = linear_similarities.argsort()[:-11:-1]
    # sort by similarity score
    linear_similarities.sort()
    a = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        a.loc[i,'index'] = str(index)
        a.loc[i,'Content'] = df_wikipedia['content'][index] ## Read File name with index from File_data DF
        a.loc[i,'File name'] = df_wikipedia['filename'][index] ## Read File name with index from File_data DF
    for j,simScore in enumerate(linear_similarities[:-11:-1]):
        a.loc[j,'Score'] = simScore
    return a

#SearchDocument('johnson and johnson vaccines derived from murdered babies. at least 2 other vaccines are not. if you take vaccines use those.')

#SearchDocument('Michael Yeadon	is a former person that was important to someone')

SearchDocument('Together with collaborators, we founded a spinout company to develop and commercialise a nasal spray vaccine against COVID. The vaccine is based on research by our and @UniEastFinland researchers. #HelsinkiHealth #corona #COVID19 #covidvaccine')

Max Sequence Length: 512


,index,Content,File name,Score
0,139,sinds januari 2020 wordt er door meerdere orga...,COVID-19,0.497134
1,766,protocol mrna 1273 p301 pdf . moderna. protoc...,COVID-19_vaccine,0.496551
2,746,a universal coronavirus vaccine is effective a...,COVID-19_vaccine,0.490719
3,823,curevac n.v. is a german biopharmaceutical com...,CureVac,0.490441
4,824,in 2007 curevac received the innovation prize ...,CureVac,0.478521
5,764,in june 2021 a report revealed that the ub 612...,COVID-19_vaccine,0.463849
6,827,cvncov is an mrna vaccine that encodes a minim...,CureVac,0.461640
7,745,intranasal vaccines target mucosal immunity in...,COVID-19_vaccine,0.458389
8,177,one idea used to support a laboratory origin i...,COVID-19_misinformation,0.442386
9,305,international research on vaccines and medicin...,COVID-19-en,0.433409


## Scoring

### Import CovidCheck

In [ ]:
import math

covid_check = pd.read_csv('no_headings_nl_merged_clef_emnlp_downsampled_to_en - Sheet1.csv', sep=',')
covid_check['tokenized_covidcheck'] = covid_check['verify_text']

covid_check['tokenized_covidcheck']= covid_check['tokenized_covidcheck'].str.lower()
SW=['<span>', '<li>', '</li>', '<ul>', '</ul>', '<b>', '</b>', '<h4>', '</h4>', '<h3>', '</h3>', '<h2>', '</h2>', '<h1>', '</h1>', '</link>', '</span>', '</i>']
for sw in SW:
    covid_check.tokenized_covidcheck=covid_check.tokenized_covidcheck.replace(to_replace=sw,value='',regex=True)

print(type(covid_check['tokenized_covidcheck'][0]))
print(covid_check.tokenized_covidcheck[0])
#covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.replace(to_replace='<[^<]+?>',value=' ',regex=True)
covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.replace(to_replace='<p(.*?)>',value='<p>',regex=True)
covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.replace(to_replace='<link(.*?)>',value=' ',regex=True)
covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.replace(to_replace='<span(.*?)>',value=' ',regex=True)
covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.replace(to_replace='<i(.*?)>',value=' ',regex=True)
covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.replace(to_replace='[!"#$%&\'()*+,:;=?@[\\]^_`{|}~]',value=' ',regex=True)
covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.replace(to_replace='-',value=' ',regex=True)
#covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.replace(to_replace='\s+',value=' ',regex=True)
#covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.replace(to_replace='\n',value=' ',regex=True)
#covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.replace(to_replace='  ',value='',regex=True)
covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.apply(lambda x:x.strip() if type(x) == str else x)

covid_check["tokenized_covidcheck"] = covid_check["tokenized_covidcheck"].str.split("<p>")
covid_check = covid_check.explode("tokenized_covidcheck").reset_index(drop=True)
covid_check = covid_check[covid_check.tokenized_covidcheck != ' ']
covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.replace(to_replace='<[^<]+?>',value=' ',regex=True)
covid_check.tokenized_covidcheck =covid_check.tokenized_covidcheck.replace(to_replace='  ',value=' ',regex=True)
covid_check["tokenized"] = covid_check["tokenized_covidcheck"]

print(covid_check['tokenized_covidcheck'][27])


<class 'float'>
nan
nan


### Final Scoring

In [ ]:
import scipy
import statistics

index = 0
matches = 0
cosine_list = []

def first_last_words(cur_list):
    fl_sentence = ''
    fl_list = []
    for sentence in cur_list:
      sent_list = sentence.split()
      if len(sent_list) > 3:
        fl_sentence = '{} {} {} {}'.format(sent_list[0], sent_list[1], sent_list[-2], sent_list[-1])
      elif len(sent_list) > 1:
        fl_sentence = sent_list[0]
      else:
        fl_sentence = None
      fl_list.append(fl_sentence)
    return fl_list

def similarity_huggingface(query):
  # Calculatte similarity for huggingface models
  Q_Train = model.encode(q)
  linear_similarities = scipy.spatial.distance.cdist(Q_Train, embeddings, "cosine")[0]
  linear_similarities = 1 - linear_similarities
  return linear_similarities

def similarity_tfhub(query):
  # Calculatte similarity for Tensorflow Hub models
  Q_Train =embed(query)
  #linear_similarities = linear_kernel(Q_Train, loadedmodel).flatten()
  linear_similarities = scipy.spatial.distance.cdist(Q_Train, loadedmodel, "cosine")[0]
  linear_similarities = 1 - linear_similarities
  return linear_similarities

def tfidf_similarity():
  index = 0
  matches = 0
  for checkworthy in covid_check['class_label']:
    correct_docs = []
    if checkworthy == 1:
      # input check-worthy message
      query = covid_check['tweet_text'][index]
      preprocessed_query = re.sub("\W+", " ", query).strip()
      tokens = word_tokenize(str(preprocessed_query))

      q_df = pd.DataFrame(columns=['q_clean'])
      q_df.loc[0,'q_clean'] =tokens
      q_df['q_clean'] =wordLemmatizer(q_df.q_clean)
      q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
      q_df=q_df.replace(to_replace ="'", value = '', regex = True)
      q_df=q_df.replace(to_replace =" ", value = '', regex = True)
      q_df=q_df.replace(to_replace ='\]', value = '', regex = True)
      
      # check if the correct document exists
      correct_text = ''
      if type(covid_check['verify_url'][index]) == str:
        correct_text = covid_check['tokenized_covidcheck'][index].split('\n\n')
        correct_text = [re.sub('\s+', ' ', _) for _ in correct_text]
        correct_text = [re.sub('  ', '', _) for _ in correct_text]
      
      d_cosines = []
      query_vector = gen_vector_T(q_df['q_clean'])
      for d in tfidf_tran.A:
        d_cosines.append(cosine_sim(query_vector, d))
      # top 10 matches
      Top_index_doc = np.array(d_cosines).argsort()[-3:][::-1]
      print(len(Top_index_doc))
      # Check if retrieved top 10 contains correct document
      #print("top docs: ", Top_index_doc)
      predicted_content = [df_wikipedia['content'][index] for index in Top_index_doc]
      #print(predicted_docs)
      if max(np.array(d_cosines)) < 0.18:
        predicted_content = ['' for i in predicted_content]

      predicted_content = first_last_words(predicted_content)
      correct_text = first_last_words(correct_text)

      flat_cosines = np.array(d_cosines).flatten()
      flat_cosines.sort()
      flat_cosines = flat_cosines[-10:]
      cosine_list.extend(flat_cosines)

      # Check if the correct answer is in the prediction
      if len(correct_text) > 0:
        for sentence in correct_text:
          if sentence in predicted_content:
            #print(len(correct_text))
            matches += 1
      else:
        if None in predicted_content:
          for i in predicted_content:
            matches += 1
    index += 1
  print("Amount correct: ", matches)

# tfidf_similarity()

for checkworthy in covid_check['class_label']:
  correct_text = []
  if checkworthy == 1:
    # input check-worthy message
    q =[covid_check['tweet_text'][index]]
    
    # check if the correct document exists
    if type(covid_check['tokenized_covidcheck'][index]) == str:
      correct_text = covid_check['tokenized_covidcheck'][index].split('\n\n')
      correct_text = [re.sub('\s+', ' ', _) for _ in correct_text]
      correct_text = [re.sub('  ', '', _) for _ in correct_text]
    else:
      print(type(covid_check['tokenized_covidcheck'][index]))

    linear_similarities = similarity_huggingface(q)
    # top 10 matches
    Top_index_doc = linear_similarities.argsort()[:-11:-1]
    # Check if retrieved top 10 contains correct document
    predicted_content = [df_wikipedia['content'][index] for index in Top_index_doc]

    #print(max(linear_similarities))
    if max(linear_similarities) < 0.37:
      predicted_content = ['' for i in predicted_content]
      #print(predicted_content)
    #print(linear_similarities[:10])

    # Check if the correct answer is in the prediction
    # print(correct_text)
    # print(predicted_content)
    # print('--------------')

    linear_similarities.sort()
    linear_similarities = linear_similarities[-10:]
    cosine_list.extend(linear_similarities)
    predicted_content = first_last_words(predicted_content)
    correct_text = first_last_words(correct_text)

    # Check if the correct answer is in the prediction
    if len(correct_text) > 0:
      for sentence in correct_text:
        if sentence in predicted_content:
          print('hoi')
          matches += 1
    else:
      if None in predicted_content:
        for i in predicted_content:
          matches += 1
  index += 1

print(len(cosine_list))
print(statistics.mean(cosine_list))
print('matches: ',matches)

<class 'float'>
hoi
hoi
hoi
<class 'float'>
hoi
<class 'float'>
hoi
<class 'float'>
hoi
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
hoi
hoi
<class 'float'>
<class 'float'>
<class 'float'>
490
0.3697380503060291
matches:  58
